In [48]:
# !pip install --upgrade pip
# !pip install pandas
# !pip install sodapy

# Data Profiling

Analyzing a subset of data to understand its characteristics, guiding the design of an efficient data pipeline.

## Data Extraction

In [49]:
import pandas as pd
from sodapy import Socrata
import json
from datetime import datetime, timedelta

In [50]:
with open('config.json') as f:
  config = json.load(f)

AppToken = config['app_token']
UserName = config['user_name']
Password = config["password"]

In [51]:
client = Socrata("data.iowa.gov",
                 AppToken,
                 username = UserName,
                 password = Password,
                 timeout=30)

In [52]:
col_selected = 'invoice_line_no, date, store, name, city, zipcode, county, category, category_name, vendor_no, vendor_name, itemno, im_desc, state_bottle_cost, state_bottle_retail, sale_bottles'
col_list = ['invoice_line_no', 'date', 'store', 'name', 'city', 'zipcode', 'county', 'category', 'category_name', 'itemno', 'im_desc', 'state_bottle_cost', 'state_bottle_retail', 'sale_bottles']
len(col_list)

14

In [53]:
start_date = "2023-06-01"
end_date = datetime.now().strftime("%Y-%m-%dT%H:%M:%S.%f")[:-3]  

total_rows = []  
limit = 5000  
offset = 0  
more_data = True  

while more_data:
    results = client.get("m3tr-qhgy",
                         select=col_selected, 
                         where=f"LOWER(name) LIKE '%hy-vee%' AND date >= '{start_date}' AND date < '{end_date}'", 
                         limit=limit, 
                         offset=offset)
    total_rows.extend(results)  

    if len(results) < limit:
        more_data = False
    else:
        offset += limit 

df = pd.DataFrame.from_records(total_rows)
print(f"Rows from {start_date} to {end_date}: {df.shape[0]}")

Rows from 2023-06-01 to 2024-01-02T18:42:13.578: 425024


In [54]:
df = df.sample(frac=0.1)
df.head()

,invoice_line_no,date,store,name,city,zipcode,county,category,category_name,vendor_no,vendor_name,itemno,im_desc,state_bottle_cost,state_bottle_retail,sale_bottles
94102,INV-60333400010,2023-07-14T00:00:00.000,2669,HY-VEE / CORYDON,CORYDON,50060,WAYNE,1071000,COCKTAILS/RTD,035,BACARDI USA INC,56856,BACARDI BAHAMA MAMA RTS,9.53,14.30,1
233479,INV-62760100042,2023-09-28T00:00:00.000,2651,HY-VEE / WAVERLY,WAVERLY,50677,BREMER,1032100,IMPORTED VODKAS,035,BACARDI USA INC,34422,GREY GOOSE,19.00,28.50,6
157015,INV-61332900001,2023-08-17T00:00:00.000,2538,HY-VEE FOOD STORE #3 (1866) / WATERLOO,WATERLOO,50702,BLACK HAWK,1081600,WHISKEY LIQUEUR,421,SAZERAC COMPANY INC,100413,FIREBALL CINNAMON WHISKEY PARTY BUCKET,51.60,77.40,1
92811,INV-60269800094,2023-07-13T00:00:00.000,2572,HY-VEE FOOD STORE / CEDAR FALLS,CEDAR FALLS,50613,BLACK HAWK,1031100,AMERICAN VODKAS,434,LUXCO INC,36305,HAWKEYE VODKA PET,3.50,5.25,12
325567,INV-64329600031,2023-11-15T00:00:00.000,2558,HY-VEE FOOD STORE (1433) / MOUNT PLEASANT,MOUNT PLEASANT,52641,HENRY,1011200,STRAIGHT BOURBON WHISKIES,306,MISSISSIPPI RIVER DISTILLING COMPANY LLC,27479,CODY ROAD BOURBON,19.17,28.76,6


## Data Transformation

### 1.1. Removing Duplicate Rows

In [55]:
num_dup = df.duplicated().sum()
perc_dup = num_dup / df.shape[0] * 100 
print(f'Number of duplicate rows: {num_dup}; Percentage of duplicate rows: {perc_dup:.2f}%')

Number of duplicate rows: 6; Percentage of duplicate rows: 0.01%


In [56]:
# Despite the potential for duplicates arising from reasons other than data entry errors, 
# I'm opting to remove them as they constitute a minimal proportion of the dataset (only 0.02%). 
# This decision is based on the assumption that the impact on data integrity will be negligible while simplifying further analysis.

df = df.drop_duplicates()
num_dup = df.duplicated().sum()
perc_dup = num_dup / df.shape[0] * 100 
print(f'Number of duplicate rows: {num_dup}; Percentage of duplicate rows: {perc_dup:.2f}%')

Number of duplicate rows: 0; Percentage of duplicate rows: 0.00%


### 1.2. Handelling Missing Values

In [57]:
def missing_values_table(df):
    if not isinstance(df, pd.DataFrame):
        raise ValueError("Input is not a pandas DataFrame")

    num_na = df.isnull().sum()
    perc_na = (num_na / len(df)) * 100

    df_na = pd.DataFrame({
        'Missing Values': num_na,
        '% of Total Values': perc_na
    })

    df_na = df_na[df_na['Missing Values'] != 0]

    return df_na

missing_values_table(df)

,Missing Values,% of Total Values


Although the current sample shows no missing values, they may be underrepresented in the full dataset. 

These rows, containing key sales data, are vital for our analysis. 
Hence, we will retain them by imputing missing values with placeholders to preserve critical insights. 

However, records lacking essential data such as price, sales volume, dates, and index will be excluded, 
as their absence renders them unsuitable for meaningful analysis.

In [58]:
df.columns

Index(['invoice_line_no', 'date', 'store', 'name', 'city', 'zipcode', 'county',
       'category', 'category_name', 'vendor_no', 'vendor_name', 'itemno',
       'im_desc', 'state_bottle_cost', 'state_bottle_retail', 'sale_bottles'],
      dtype='object')

In [59]:
dimension_cols = ['store', 'name', 'city', 'zipcode', 'county', 'category', 'category_name', 'vendor_no', 'vendor_name', 'itemno', 'im_desc']

In [60]:
def generate_placeholder_dict(df, cols):
    placeholder_dict = {}
    for col in cols:
        try:
            int_col = df[col].dropna().astype(int)
            mode_digit_count = int_col.astype(str).apply(len).mode()[0]
            placeholder_dict[col] = int('9' * mode_digit_count)
        except ValueError:
            placeholder_dict[col] = 'UNKNOWN'
    return placeholder_dict

# Assuming 'df' is your DataFrame
dimension_cols = ['store', 'name', 'city', 'zipcode', 'county', 'category', 'category_name', 'vendor_no', 'vendor_name', 'itemno', 'im_desc']
placeholders = generate_placeholder_dict(df, dimension_cols)

placeholders

{'store': 9999,
 'name': 'UNKNOWN',
 'city': 'UNKNOWN',
 'zipcode': 99999,
 'county': 'UNKNOWN',
 'category': 9999999,
 'category_name': 'UNKNOWN',
 'vendor_no': 999,
 'vendor_name': 'UNKNOWN',
 'itemno': 99999,
 'im_desc': 'UNKNOWN'}

In [61]:
df = df.fillna(value=placeholders)
missing_values_table(df)

,Missing Values,% of Total Values


### 2.3. Checking Data Type

Verify if specific columns, such as indexes, are convertible to integers and if the date column can be formatted as datetime for compatibility with corresponding MySQL server data types

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42496 entries, 94102 to 356915
Data columns (total 16 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   invoice_line_no      42496 non-null  object
 1   date                 42496 non-null  object
 2   store                42496 non-null  object
 3   name                 42496 non-null  object
 4   city                 42496 non-null  object
 5   zipcode              42496 non-null  object
 6   county               42496 non-null  object
 7   category             42496 non-null  object
 8   category_name        42496 non-null  object
 9   vendor_no            42496 non-null  object
 10  vendor_name          42496 non-null  object
 11  itemno               42496 non-null  object
 12  im_desc              42496 non-null  object
 13  state_bottle_cost    42496 non-null  object
 14  state_bottle_retail  42496 non-null  object
 15  sale_bottles         42496 non-null  object
dtypes: o

#### 2.3.1. Datetime Columns

In [63]:
try:
    pd.to_datetime(df['date'])
    print("The 'date' column is in a suitable format for SQL date type.")
except (ValueError, TypeError):
    print("The 'date' column is NOT in a suitable format for SQL date type.")

The 'date' column is in a suitable format for SQL date type.


#### 2.3.2. Integer Columns

In [64]:
convertible_to_int = {}

for col in df.columns:
    if df[col].dtype == 'object' and col != 'date':
        converted_col = pd.to_numeric(df[col], errors='coerce')
        if not converted_col.isna().any():
            convertible_to_int[col] = 'int'
            print(f"The column '{col}' can be fully converted to integers.")
        else:
            print(f"The column '{col}' cannot be fully converted to integers.")

print(convertible_to_int)

The column 'invoice_line_no' cannot be fully converted to integers.
The column 'store' can be fully converted to integers.
The column 'name' cannot be fully converted to integers.
The column 'city' cannot be fully converted to integers.
The column 'zipcode' can be fully converted to integers.
The column 'county' cannot be fully converted to integers.
The column 'category' can be fully converted to integers.
The column 'category_name' cannot be fully converted to integers.
The column 'vendor_no' can be fully converted to integers.
The column 'vendor_name' cannot be fully converted to integers.
The column 'itemno' can be fully converted to integers.
The column 'im_desc' cannot be fully converted to integers.
The column 'state_bottle_cost' can be fully converted to integers.
The column 'state_bottle_retail' can be fully converted to integers.
The column 'sale_bottles' can be fully converted to integers.
{'store': 'int', 'zipcode': 'int', 'category': 'int', 'vendor_no': 'int', 'itemno': 'i

### 2.4. Data Validation

In [65]:
def convert_to_numeric(df, col):
    # Convert column to float first
    df[col] = df[col].astype(float)
    # If the float value is equal to its integer conversion, change to int
    df[col] = df[col].apply(lambda x: int(x) if x == int(x) else x)

columns_to_convert = ['state_bottle_cost', 'state_bottle_retail', 'sale_bottles']
for col in columns_to_convert:
    convert_to_numeric(df, col)

In [66]:
df.describe()

,state_bottle_cost,state_bottle_retail,sale_bottles
count,42496.000000,42496.000000,42496.000000
mean,13.706628,20.562130,11.704655
std,12.535955,18.804342,44.339010
min,0.970000,1.460000,-24.000000
25%,7.250000,10.880000,3.000000
50%,10.970000,16.460000,6.000000
75%,16.000000,24.000000,12.000000
max,300.000000,450.000000,5400.000000


In [67]:
# Check the percentage of rows where 'sale_bottles' is negative
len(df[df['sale_bottles']<0])/len(df) * 100

0.13883659638554216

**Note**: While negative sales data typically indicate refunds or returns, our analysis will omit these records to maintain data integrity. 

This decision is based on the focus of our study, which requires consistent and positive sales figures to accurately assess trends and patterns.


In [68]:
df=df[df['sale_bottles']>=0]
df.describe()

,state_bottle_cost,state_bottle_retail,sale_bottles
count,42437.000000,42437.000000,42437.000000
mean,13.705425,20.560327,11.733464
std,12.534942,18.802824,44.362345
min,0.970000,1.460000,1.000000
25%,7.250000,10.880000,3.000000
50%,10.970000,16.460000,6.000000
75%,16.000000,24.000000,12.000000
max,300.000000,450.000000,5400.000000
